In [1]:
import pandas as pd

df = pd.read_csv("langchain_docs_modules.csv")

In [3]:
from getpass import getpass

OPENAI_API_KEY = getpass()

In [4]:
import os

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [19]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document

documents = []
for index, row in df.iterrows():    
    documents.append(Document(page_content=row["text"],metadata={"source": row["source"]}))

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

db = Chroma.from_documents(documents, embeddings)
    

Using embedded DuckDB without persistence: data will be transient


In [87]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
prompt_template = """Use the following pieces of context from the LangChain documentation to write code based on the question below. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Respond with Python code including importing all required packages:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}
model = ChatOpenAI(model_name='gpt-3.5-turbo') # switch to 'gpt-4'
qa = RetrievalQA.from_chain_type(llm=model, chain_type="stuff", retriever=db.as_retriever(), chain_type_kwargs=chain_type_kwargs, return_source_documents=True)

In [99]:
# query = "How do I write a custom prompt template in LangChain? Provide an example" 
query = "How do I use LLM chain?" 
result = qa(query)

In [101]:
print(result["result"])

```
from langchain import PromptTemplate, OpenAI, LLMChain

# Define a prompt template
prompt_template = "What is a good name for a company that makes {product}?"

# Initialize the OpenAI LLM model
llm = OpenAI(temperature=0)

# Initialize the LLMChain with the prompt template and the OpenAI LLM model
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)

# Use the LLMChain to generate a company name based on the input product
llm_chain("colorful socks")
```


In [102]:
from langchain import PromptTemplate, OpenAI, LLMChain

# Define a prompt template
prompt_template = "What is a good name for a company that makes {product}?"

# Initialize the OpenAI LLM model
llm = OpenAI(temperature=0)

# Initialize the LLMChain with the prompt template and the OpenAI LLM model
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)

# Use the LLMChain to generate a company name based on the input product
llm_chain("colorful socks")

{'product': 'colorful socks', 'text': '\n\nSocktastic!'}

In [98]:
result

{'query': 'How do I write a custom prompt template in LangChain? Provide an example',
 'result': 'from langchain.prompts.prompt import PromptTemplate\n\n# Define the custom prompt template\nprompt_template = """Given the function name \'{function_name}\', write an English language explanation of the function.\nFunction Definition:\n{function_source}\n"""\n\n# Define the input variables for the prompt template\ninput_variables = ["function_name", "function_source"]\n\n# Create a PromptTemplate object with the custom prompt template and input variables\ncustom_prompt = PromptTemplate(template=prompt_template, input_variables=input_variables)\n\n# Create an LLMChain object with the custom prompt template\nllm = OpenAI(temperature=0)\nllm_chain = LLMChain(prompt=custom_prompt, llm=llm, verbose=True)',
 'source_documents': [Document(page_content="# How to create a custom prompt template  Let's suppose we want the LLM to generate English language explanations of a function given its name. To

In [91]:
from langchain.prompts.prompt import PromptTemplate

# Define the custom prompt template
prompt_template = """Given the function name '{function_name}', write an English language explanation of the function.
Function Definition:
{function_source}
"""

# Define the input variables for the prompt template
input_variables = ["function_name", "function_source"]

# Create a PromptTemplate object with the custom prompt template and input variables
custom_prompt = PromptTemplate(template=prompt_template, input_variables=input_variables)

# Create an LLMChain object with the custom prompt template
llm = OpenAI(temperature=0)
llm_chain = LLMChain(prompt=custom_prompt, llm=llm, verbose=True)

In [96]:
src = """
def sum(xs):
    count = 0
    for x in xs:
        count += x
    return count
"""

name = "sum"

input = {
    "function_name": name,
    "function_source": src
}

In [97]:
llm_chain(input)



> Entering new LLMChain chain...
Prompt after formatting:
Given the function name 'sum', write an English language explanation of the function.
Function Definition:

def sum(xs):
    count = 0
    for x in xs:
        count += x
    return count



> Finished chain.


{'function_name': 'sum',
 'function_source': '\ndef sum(xs):\n    count = 0\n    for x in xs:\n        count += x\n    return count\n',
 'text': '\nThe sum function takes a list of numbers (xs) as an argument and returns the sum of all the numbers in the list. It does this by looping through each number in the list and adding it to a running total, which is then returned as the result.'}